In [42]:
%matplotlib auto
import numpy as np
import matplotlib.pyplot as plt
import csv

# compute prob of tamplate
def prob(n, cov_mat, N):
    det = np.linalg.det(cov_mat)
    cov_mat_inv = np.linalg.pinv(cov_mat)
    return 1/np.sqrt(det * pow(2*np.pi,N))*np.exp(-0.5*np.dot(np.dot(n.T,cov_mat_inv), n))
    

Using matplotlib backend: TkAgg


In [17]:
# plot ldr traces with different HW(0-32) in two model

# HWmodel
for i in range(0, 33):
    trc = np.loadtxt(f"ldr_profile/HW_trc/hw{i}.trc")
    plt.plot(trc)# , label = f"{i}")
plt.legend()
plt.show()

# powermodel
plt.figure()
for i in range(0, 33):
    trc = np.loadtxt(f"ldr_profile/HW_trc/hw{i}_trc.trc")
    plt.plot(trc, label = f"{i}")
plt.legend()
plt.show()


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [7]:
# distribution of traces with different length
tamplate_num = 33
profiled_trc_ldr_num = 500
cnt_dis = [[],[],[]]
for i in range(0, 33):
    cnt = [0,0,0]
    for j in range(0, profiled_trc_ldr_num):
        trc = np.loadtxt(f"ldr_profile/hw{i}/{j+1}.trc")
        if(len(trc)==1011):
            cnt[0]+=1
        elif len(trc)==1015:
            cnt[1]+=1
        elif len(trc)==1019:
            cnt[2]+=1
    cnt_dis[0].append(cnt[0])
    cnt_dis[1].append(cnt[1])
    cnt_dis[2].append(cnt[2])
    # print(f'{i} {cnt[0]} {cnt[1]} {cnt[2]}') 
plt.plot(cnt_dis[0], label='1011')
plt.plot(cnt_dis[1], label='1015')
plt.plot(cnt_dis[2], label='1019')
plt.legend()
plt.xlabel('HW')
plt.ylabel('trace number')
plt.show()

In [13]:
# details of different length
trc1 = np.loadtxt(f"ldr_profile/hw{26}/{3}.trc")
trc2 = np.loadtxt(f"ldr_profile/hw{26}/{2}.trc")
trc3 = np.loadtxt(f"ldr_profile/hw{26}/{1}.trc")
plt.plot(trc1, label="1011")
plt.plot(trc3, label="1019")
plt.plot(trc2, label="1015")

plt.legend()
plt.show()

In [109]:
# load profiled traces

tamplate_num = 33
profiled_trc_ldr_num = 500

POI_lst = [38, 40, 82] 
POI_num = 3
profiled_trc_ldr = np.zeros([tamplate_num, profiled_trc_ldr_num, POI_num])
for i in range(0, tamplate_num):
    for j in range(0, profiled_trc_ldr_num):
        trc = np.loadtxt(f"ldr_profile/hw{i}/{j+1}.trc")
        for k in range(0, POI_num):
            profiled_trc_ldr[i][j][k] = trc[POI_lst[k]] + np.random.normal(0, 0.001)

In [37]:
# show the diff between traces from hw13 and hw14
plt.figure()
for i in range(0, 500):
    trc = np.loadtxt(f"ldr_profile/hw13/{i+1}.trc")
    plt.plot(trc, label='hw13', color="red")

for i in range(0, 500):
    trc = np.loadtxt(f"ldr_profile/hw14/{i+1}.trc")
    plt.plot(trc, label="hw14", color='blue')

plt.show()

In [110]:
# build tamplates for load s[i]

# compute average trace Mi for each tamplate
profiled_trc_ldr_mean = np.zeros([tamplate_num, POI_num])
for i in range(0, tamplate_num):
    profiled_trc_ldr_mean[i] = np.mean(profiled_trc_ldr[i], axis=0)

# compute noise vector for all L sample traces
noise_vec_ldr = np.zeros([tamplate_num, profiled_trc_ldr_num, POI_num])
for i in range(0, tamplate_num):
    for j in range(0, profiled_trc_ldr_num):
        for k in range(0, POI_num):
            noise_vec_ldr[i][j][k] =  profiled_trc_ldr[i][j][k] - profiled_trc_ldr_mean[i][k]

# compute cov matrix of corresponding noise vector
cov_mat_ldr = np.zeros([tamplate_num, POI_num, POI_num])
for i in range(0, tamplate_num):
    cov_mat_ldr[i] = np.cov(noise_vec_ldr[i], rowvar=False)

In [71]:
# test tamplate accuracy of each tamplate
res = [['HW1_hit', 'HW5_hit']]
ave_HW1 = 0
ave_HW5 = 0


# load test traces from file
for test_hw in range(0, 33):
    test_trc_num = 1000
    test_trc_ldr = np.zeros([test_trc_num, POI_num])
    for i in range(0, test_trc_num):
        trc = np.loadtxt(f"hw_ldr_test/hw{test_hw}/{i+1}.trc")
        for k in range(0, POI_num):
            test_trc_ldr[i][k] = trc[POI_lst[k]]

    # test tamplate for load s1[i]
    poss_value_lst = np.zeros([test_trc_num, tamplate_num])
    for k in range(0, test_trc_num):
        nv = np.zeros([tamplate_num, POI_num])
        for i in range(0, tamplate_num):
            for j in range(0, POI_num):
                nv[i][j] = test_trc_ldr[k][j] - profiled_trc_ldr_mean[i][j]
            poss_value_lst[k][i] = prob(nv[i], cov_mat_ldr[i], POI_num)
    
    # sort Hamming weights according to probability
    predict_res_lst = np.zeros([test_trc_num, tamplate_num])
    for k in range(0, test_trc_num):
        val = poss_value_lst[k]
        hw = np.array(list(range(0, tamplate_num))) 
        index = np.argsort(val)
        predict_res_lst[k] = hw[index]

    # compute the accuracy of tamplates
    HW1_hit = 0
    HW5_hit = 0
    for k in range(0, test_trc_num):
        if predict_res_lst[k][-1] == test_hw:
            HW1_hit += 1
            ave_HW1 += 1
        if test_hw in predict_res_lst[k][-5:]:
            HW5_hit += 1
            ave_HW5 += 1 
    print(f'HW{test_hw}: HW1_rate = {HW1_hit/test_trc_num} HW5_rate = {HW5_hit/test_trc_num}')
    res.append([test_hw, HW1_hit/test_trc_num, HW5_hit/test_trc_num])
print(f"average: HW1_rate = {ave_HW1/(test_trc_num*33)} HW5_rate = {ave_HW5/(test_trc_num*33)}")

csvfile = open("hw_ldr_rate.csv", 'w', newline='')
csvwriter = csv.writer(csvfile)
for row in res:
    csvwriter.writerow(row)
csvfile.close()



HW0: HW1_rate = 1.0 HW5_rate = 1.0
HW1: HW1_rate = 0.979 HW5_rate = 1.0
HW2: HW1_rate = 0.844 HW5_rate = 1.0
HW3: HW1_rate = 0.728 HW5_rate = 1.0
HW4: HW1_rate = 0.693 HW5_rate = 1.0
HW5: HW1_rate = 0.622 HW5_rate = 1.0
HW6: HW1_rate = 0.658 HW5_rate = 1.0
HW7: HW1_rate = 0.594 HW5_rate = 1.0
HW8: HW1_rate = 0.533 HW5_rate = 1.0
HW9: HW1_rate = 0.544 HW5_rate = 1.0
HW10: HW1_rate = 0.542 HW5_rate = 1.0
HW11: HW1_rate = 0.509 HW5_rate = 0.998
HW12: HW1_rate = 0.545 HW5_rate = 0.994
HW13: HW1_rate = 0.513 HW5_rate = 0.997
HW14: HW1_rate = 0.499 HW5_rate = 0.995
HW15: HW1_rate = 0.363 HW5_rate = 0.997
HW16: HW1_rate = 0.373 HW5_rate = 0.997
HW17: HW1_rate = 0.391 HW5_rate = 1.0
HW18: HW1_rate = 0.458 HW5_rate = 0.993
HW19: HW1_rate = 0.493 HW5_rate = 0.993
HW20: HW1_rate = 0.52 HW5_rate = 0.999
HW21: HW1_rate = 0.546 HW5_rate = 0.998
HW22: HW1_rate = 0.524 HW5_rate = 0.999
HW23: HW1_rate = 0.549 HW5_rate = 1.0
HW24: HW1_rate = 0.563 HW5_rate = 1.0
HW25: HW1_rate = 0.567 HW5_rate = 1.0
HW2

In [111]:
# test ldr s[i] tamplates on 10 Case (1000 rnd number) 
ave_HW1 = 0
ave_HW5 = 0
f = open("ldr_test/rnd_ldr_test_hw.txt", "r") # read correct hw of every case
flines = f.readlines()

# load test traces from file
Kase = 10
for kase in range(0, Kase):
    test_trc_num = 1000
    test_trc_ldr = np.zeros([test_trc_num, POI_num])
    for i in range(0, test_trc_num):
        trc = np.loadtxt(f"ldr_test/Case{kase}/{i+1}.trc")
        for k in range(0, POI_num):
            test_trc_ldr[i][k] = trc[POI_lst[k]]

    # compute the probability of test traces
    poss_value_lst = np.zeros([test_trc_num, tamplate_num])
    for k in range(0, test_trc_num):
        nv = np.zeros([tamplate_num, POI_num])
        for i in range(0, tamplate_num):
            for j in range(0, POI_num):
                nv[i][j] = test_trc_ldr[k][j] - profiled_trc_ldr_mean[i][j]
            poss_value_lst[k][i] = prob(nv[i], cov_mat_ldr[i], POI_num)
    
    # sort Hamming weights according to probability
    predict_res_lst = np.zeros([test_trc_num, tamplate_num])
    for k in range(0, test_trc_num):
        val = poss_value_lst[k]
        hw = np.array(list(range(0, tamplate_num))) 
        index = np.argsort(val)
        predict_res_lst[k] = hw[index]

    # hw answer of the case
    ans = flines[2*kase+1].split(",")
    ans = ans[0:-1]
    ans = [int(i) for i in ans]

    # compute the accuracy of tamplates
    HW1_hit = 0
    HW5_hit = 0
    for k in range(0, test_trc_num):
        if predict_res_lst[k][-1] == ans[k]:
            HW1_hit += 1
            ave_HW1 += 1
        if ans[k] in predict_res_lst[k][-5:]:
            HW5_hit += 1
            ave_HW5 += 1 
    print(f'Case{kase}: HW1_rate = {HW1_hit/test_trc_num} HW5_rate = {HW5_hit/test_trc_num}')
    # res.append([test_hw, HW1_hit/test_trc_num, HW5_hit/test_trc_num, HW6_hit/test_trc_num,HW10_hit/test_trc_num])
print(f"average: HW1_rate = {ave_HW1/(test_trc_num*Kase)} HW5_rate = {ave_HW5/(test_trc_num*Kase)}")
'''
csvfile = open("ldr_rate.csv", 'w', newline='')
csvwriter = csv.writer(csvfile)
for row in res:
    csvwriter.writerow(row)
csvfile.close()
'''


Case0: HW1_rate = 0.339 HW5_rate = 0.967
Case1: HW1_rate = 0.365 HW5_rate = 0.98
Case2: HW1_rate = 0.329 HW5_rate = 0.982
Case3: HW1_rate = 0.353 HW5_rate = 0.982
Case4: HW1_rate = 0.337 HW5_rate = 0.977
Case5: HW1_rate = 0.329 HW5_rate = 0.987
Case6: HW1_rate = 0.345 HW5_rate = 0.983
Case7: HW1_rate = 0.334 HW5_rate = 0.974
Case8: HW1_rate = 0.33 HW5_rate = 0.977
Case9: HW1_rate = 0.322 HW5_rate = 0.98
average: HW1_rate = 0.3383 HW5_rate = 0.9789


'\ncsvfile = open("ldr_rate.csv", \'w\', newline=\'\')\ncsvwriter = csv.writer(csvfile)\nfor row in res:\n    csvwriter.writerow(row)\ncsvfile.close()\n'

In [113]:
# load profiled traces of x[i] from file

tamplate_num = 33
profiled_trc_str_num = 1000

POI_lst = [183, 185, 207] #[178, 183,185,187, 207]
POI_num = 3

profiled_trc_str = np.zeros([tamplate_num, profiled_trc_str_num, POI_num])
for i in range(0, tamplate_num):
    for j in range(0, profiled_trc_str_num):
        trc = np.loadtxt(f"str_profile/hw{i}/{j+1}.trc")
        for k in range(0, POI_num):
            profiled_trc_str[i][j][k] = trc[POI_lst[k]] + np.random.normal(0, 0.001)


In [19]:
# plot str traces with different HW(0-32) in two model
plt.figure()
for i in range(0, 33):
    trc = np.loadtxt(f"str_profile/HW_trc/hw{i}.trc")
    plt.plot(trc)# , label = f"{i}")
plt.legend()
plt.show()

plt.figure()
for i in range(0, 33):
    trc = np.loadtxt(f"str_profile/HW_trc/hw{i}_trc.trc")
    plt.plot(trc, label = f"{i}")
plt.legend()
plt.show()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [ ]:
# show diff between hw0 and hw1
plt.figure()
for i in range(0, 200):
    trc = np.loadtxt(f"str_profile/hw0/{i+1}.trc")
    plt.plot(trc, label = f"{i}")
plt.legend()
plt.show()
plt.figure()
for i in range(0, 200):
    trc = np.loadtxt(f"str_profile/hw1/{i+1}.trc")
    plt.plot(trc, label = f"{i}")
plt.legend()
plt.show()

In [114]:
# build tamplate for str x[i]

# average trace of each tamplate
# compute average trace Mi
profiled_trc_str_mean = np.zeros([tamplate_num, POI_num])
for i in range(0, tamplate_num):
    profiled_trc_str_mean[i] = np.mean(profiled_trc_str[i], axis=0)

# compute noise vector for all L sample traces
noise_vec_str = np.zeros([tamplate_num, profiled_trc_str_num, POI_num])
for i in range(0, tamplate_num):
    for j in range(0, profiled_trc_str_num):
            for k in range(0, POI_num):
                noise_vec_str[i][j][k] =  profiled_trc_str[i][j][k] - profiled_trc_str_mean[i][k]

# compute cov matrix of corresponding noise vector
cov_mat_str = np.zeros([tamplate_num, POI_num, POI_num])
for i in range(0, tamplate_num):
    cov_mat_str[i] = np.cov(noise_vec_str[i], rowvar=False)

In [22]:
# test str x[i] tamplates on 10 Case (1000 rnd number) 
ave_HW1 = 0
ave_HW5 = 0
f = open("str_test/rnd_str_test_hw.txt", "r")
fv = open("str_test/rnd_str_test.txt", "r")
fvlines = fv.readlines()
flines = f.readlines()

# load test traces from file
Kase = 10
for kase in range(0, Kase):
    test_trc_num = 1000
    test_trc_data = np.zeros([test_trc_num, POI_num])
    for j in range(0, test_trc_num):
        trc = np.loadtxt(f"str_test/Case{kase}/{j + 1}.trc")
        for k in range(0, POI_num):
            test_trc_data[j][k] = trc[POI_lst[k]]
            
    # compute the probability of test trace 
    poss_value_lst = np.zeros([test_trc_num, tamplate_num])
    for j in range(0, test_trc_num):
        nv = np.zeros([tamplate_num, POI_num])
        for i in range(0, tamplate_num):
            for k in range(0, POI_num):
                nv[i][k] = test_trc_data[j][k] - profiled_trc_str_mean[i][k]
            poss_value_lst[j][i] = prob(nv[i], cov_mat_str[i], POI_num)

    predict_res_lst = np.zeros([test_trc_num, tamplate_num])
    for j in range(0, test_trc_num):
        val = poss_value_lst[j]
        hw = np.array(list(range(0, tamplate_num))) 
        index = np.argsort(val)
        predict_res_lst[j] = hw[index]

    ans = flines[2*kase+1].split(",")
    ans = ans[0:-1]
    ans = [int(i) for i in ans]
    
    ansv = fvlines[2*0+1].split(",")
    ansv = ansv[1:-1]
    ansv = [int(i) for i in ansv]
    
    HW1_hit = 0
    HW5_hit = 0
    for k in range(0, test_trc_num):
        if predict_res_lst[k][-1] == ans[k]:
            HW1_hit += 1
            ave_HW1 += 1
        if ans[k] in predict_res_lst[k][-5:]:
            HW5_hit += 1
            ave_HW5 += 1 
        '''
        if ans[k] not in predict_res_lst[k][-5:]:
            print(f"{k} {predict_res_lst[k][-5:]} {ans[k]} {ansv[k]}")
        '''
    print(f'Case{kase}: HW1_rate = {HW1_hit/test_trc_num} HW5_rate = {HW5_hit/test_trc_num}')
    # res.append([test_hw, HW1_hit/test_trc_num, HW5_hit/test_trc_num, HW6_hit/test_trc_num,HW10_hit/test_trc_num])
print(f"average: HW1_rate = {ave_HW1/(test_trc_num*Kase)} HW5_rate = {ave_HW5/(test_trc_num*Kase)}")

'''
csvfile = open(f"str_rate.csv", 'w', newline='')
csvwriter = csv.writer(csvfile)
for row in res:
    csvwriter.writerow(row)
csvfile.close()
'''


Case0: HW1_rate = 0.466 HW5_rate = 0.993
Case1: HW1_rate = 0.468 HW5_rate = 0.993
Case2: HW1_rate = 0.445 HW5_rate = 0.992
Case3: HW1_rate = 0.469 HW5_rate = 0.994
Case4: HW1_rate = 0.462 HW5_rate = 0.994
Case5: HW1_rate = 0.454 HW5_rate = 0.993
Case6: HW1_rate = 0.483 HW5_rate = 0.992
Case7: HW1_rate = 0.462 HW5_rate = 0.996
Case8: HW1_rate = 0.463 HW5_rate = 0.988
Case9: HW1_rate = 0.463 HW5_rate = 0.993
average: HW1_rate = 0.4635 HW5_rate = 0.9928


'\ncsvfile = open(f"str_rate.csv", \'w\', newline=\'\')\ncsvwriter = csv.writer(csvfile)\nfor row in res:\n    csvwriter.writerow(row)\ncsvfile.close()\n'

In [55]:
# plot HW distribution of range [-Q, Q] 
def HW(a):
    count = 0
    a = a & 0xFFFFFFFF
    while a:
        a &= a - 1
        count += 1
    return count
hw_cnt=np.zeros(33)
Q = 8380417
for s in range(-Q,Q+1):
    hw = HW(s)
    hw_cnt[hw]+=1
plt.plot(hw_cnt)
plt.xlabel('hw')
plt.ylabel('cnt')
plt.show()

In [115]:
# single trace 
hw_str_lst = np.zeros([4,5], dtype='float64')
prob_str_lst = np.zeros([4,5], dtype='float64')
str_pos_lst = [11,209,407,605]
trc_len = 198
for u in range(0, 4):
    test_trc_str = np.zeros([1, POI_num])
    trc = np.loadtxt(f"ldr_test/Case{0}/{7}.trc")
    trc = np.append(np.zeros([1,11]),trc[str_pos_lst[u]:str_pos_lst[u]+trc_len])
    # plt.plot(trc)

    for k in range(0, POI_num):
        test_trc_str[0][k] = trc[POI_lst[k]]

    # test tamplate for load s1[i]
    poss_value_lst = np.zeros([1, tamplate_num])

    nv = np.zeros([tamplate_num, POI_num])
    for i in range(0, tamplate_num):
        for j in range(0, POI_num):
            nv[i][j] = test_trc_str[0][j] - profiled_trc_str_mean[i][j]
        poss_value_lst[0][i] = prob(nv[i], cov_mat_str[i], POI_num)

    predict_res_lst = np.zeros([1, tamplate_num])
    val = poss_value_lst[0]
    hw = np.array(list(range(0, tamplate_num))) 
    index = np.argsort(val)
    predict_res_lst[0] = hw[index]

    hw_str_lst[u] = predict_res_lst[0][-5:]
    prob_str_lst[u] = np.array(sorted(val)[-5:], dtype='float64')
    # print(f'{predict_res_lst[0][-5:]} {sorted(val)[-5:]}')

for u in range(0, 4):
    print(f'{hw_str_lst[u]} {prob_str_lst[u]}')

[19. 23. 20. 22. 21.] [4.43887385e-09 8.19635800e-09 2.25244793e-08 2.64887802e-08
 3.47050039e-08]
[15. 21. 24. 23. 22.] [1.61558775e-10 6.46460290e-10 8.55319159e-10 3.07687688e-09
 3.15654293e-09]
[13.  9. 12. 10. 11.] [6.79680524e-11 1.67736642e-09 3.09141037e-09 1.55052598e-08
 1.88658780e-08]
[10.  6.  9.  7.  8.] [5.26668266e-12 6.51164539e-12 1.32610411e-10 2.25530099e-10
 5.61022463e-10]
